In [0]:
import numpy as np
from scipy.io import loadmat
from scipy import stats

In [0]:
dat = loadmat("/content/ion.mat")
xtr = dat['xTr']
ytr = dat['yTr']
yte = dat['yTe']
xte = dat['xTe']
xtr = xtr.reshape(xtr.shape[1],-1)
xte = xte.reshape(xte.shape[1],-1)

In [0]:
def entropysplit(xtr,ytr):
    # xtr : nxd
    # ytr : nx1
    n = xtr.shape[0]
    dim = xtr.shape[1]
    
    IG = 0 #Information Gain set to 0 initially
    final_entropy = 0
    final_cut = 0
    final_dim = 0
    lst = []
    
    ytr = ytr.reshape(-1)
    _,uniq = np.unique(ytr,return_counts = True)
    tot = np.sum(uniq)
    pk = uniq/tot
    init_entropy = -1*np.sum(pk*np.log2(pk))
    
    if(init_entropy == 0):
        return (-1,-1)
    
    
    for d in range(dim):
        x = xtr[:,d]
        ind = np.argsort(x)
        x = x[ind]
        y = ytr[ind]
        for i in range(len(x)-1):
            cut = x[i]
            ii = (x > cut)
            ynew = y[ii]
            sR = len(ynew)/n
            sL = 1 - sR
            _,c = np.unique(ynew,return_counts = True)
            tot = np.sum(c)
            pk = c/tot
            eR = -1*np.sum(pk*np.log2(pk))
            
            ii = (x <= cut)
            ynew = y[ii]
            _,c = np.unique(ynew,return_counts = True)
            tot = np.sum(c)
            pk = c/tot
            eL = -1*np.sum(pk*np.log2(pk))
            
            entropy = sR*eR + sL*eL
            lst.append(entropy)
            check = init_entropy - entropy
            
            if check > IG:
                IG = check 
                final_entropy = entropy
                final_cut = cut
                final_dim = d          
    return final_cut,final_dim

In [0]:
def evaluate(t, xte, yte): 
    # t : Tree Matrix
    # xte : nxd ---> test set
    n = xte.shape[0]
    ypred = np.zeros((n,1))
    for i in range(n):
        index = 0
        while True:
            if t[3,index] == 0:
                ypred[i] = t[0,index]
                break
            else:
                if xte[i,int(t[1,index])] > t[2,index]:
                    index = int(t[3,index])
                else:
                    index = int(t[4,index])
    
    ypred = ypred.reshape(-1)
    yte = yte.reshape(-1)
    acc = np.sum(ypred == yte)/(len(ypred))
    return (1-acc),ypred

In [0]:
def RandomForest(xtr,ytr, t, column = 0, child = 1, maxdepth = 1000, depth = 0):
  if depth > maxdepth:
    return
  d = xtr.shape[1]
  k = int(np.round(np.sqrt(d)))
  col = np.random.permutation(d)[:k]
  ytr = ytr.reshape(-1)
  mod = stats.mode(ytr)[0]
  t[0,column] = mod

  x_changed = xtr[:,col]
  cut, dim = entropysplit(x_changed, ytr)
  if(dim == -1):
    t[1,column] = 0
    t[2,column] = 0
    t[3,column] = 0
    t[4,column] = 0
    return
  dim = col[dim] # mapping back the dimention 

  t[1,column] = dim 
  t[2,column] = cut
  t[3,column] = child
  t[4,column] = child + 1 
  child = child + 2
  depth = depth + 1

  ii = xtr[:,dim] > cut
  x_greater = xtr[ii,:]
  y_greater = ytr[ii]

  ii1 = xtr[:,dim]<= cut
  x_less = xtr[ii1,:]
  y_less = ytr[ii1]

  RandomForest(x_greater, y_greater,t, int(t[3,column]),child, maxdepth, depth)
  RandomForest(x_less, y_less,t, int(t[4,column]),child, maxdepth, depth)
  return t


In [0]:
def baggingforForest(xtr,ytr, xte, yte, m = 100):
    n = xtr.shape[0]
    ytr = ytr.reshape(-1)
    for i in range(m):
        t = np.zeros((5,n*3))

        ## Making Data with replacement

        v = np.arange(n)
        ii =np.random.choice(v, size = n, replace = True)
        xnew = xtr[ii,:]
        ynew = ytr[ii]


        #####------------------------######

        # ind = v[np.isin(v,ii,invert=True)]
        # xtest = xtr[ind,:]
        # ytest = ytr[ind]
    
        t = RandomForest(xnew,ynew,t)
        err, ypred = evaluate(t,xte,yte)
    #     total_error = total_error + (err)

    # total_error = total_error/m
    # return total_error 

        if i == 0:
            conc = ypred.reshape(-1,1)
        else:
            conc = np.concatenate((conc,ypred.reshape(-1,1)), axis = 1)
    
    
    final = stats.mode(conc, axis = 1)[0]
    ytr = ytr.reshape(-1,1)
    acc = np.sum(final == yte.reshape(-1,1))/len(final)
    return (1-acc)
        

In [138]:
err = baggingforForest(xtr,ytr,xtr,ytr,130)
print("The error for the sample set is :",err*100,"%")

The error for the sample set is : 14.590747330960852 %
